In [ ]:
%matplotlib inline

Tagged Arrays
=============


This notebook introduces `moscot.solvers.TaggedArray`{.interpreted-text
role="class"}.

`moscot.solvers.TaggedArray`{.interpreted-text role="class"} stores the
data passed by the users in a unified way before it is passed to the
backend.


In [ ]:
from moscot.datasets import simulate_data
from moscot.problems.generic import FGWProblem
import scanpy as sc

import numpy as np
import pandas as pd

rng = np.random.default_rng(seed=42)
adata = simulate_data(n_distributions=2, key="batch")
sc.pp.pca(adata)
adata

We instantiate and prepare a
`moscot.problems.generic.FGWProblem`{.interpreted-text role="class"} to
demonstrate the role of `moscot.solvers.TaggedArray`{.interpreted-text
role="class"}.


In [ ]:
fgwp = FGWProblem(adata)
fgwp = fgwp.prepare(key="batch", joint_attr="X_pca", GW_x="X_pca", GW_y="X_pca")

The [:class:moscot.problems.base.OTProblem]{.title-ref} has attributes
[:attr:moscot.problems.base.OTProblem.xy]{.title-ref},
[:attr:moscot.problems.base.OTProblem.x]{.title-ref}, and
[:attr:moscot.problems.base.OTProblem.y]{.title-ref}, storing the data
for the linear and quadratic terms, respectively. These attributes are
all of type `moscot.solvers.TaggedArray`{.interpreted-text
role="class"}.


In [ ]:
print(fgwp["0", "1"].x)
print(fgwp["0", "1"].y)
print(fgwp["0", "1"].xy)


# Each :class:`moscot.solvers.TaggedArray` has attributes
# :attr:`moscot.solvers.TaggedArray.data_src`, :attr:`moscot.solvers.TaggedArray.data_tgt`,
# :attr:`moscot.solvers.TaggedArray.cost`, and :attr:`moscot.solvers.TaggedArray.tag`.

print(fgwp["0", "1"].xy.data_src)
print(fgwp["0", "1"].xy.data_tgt)
print(fgwp["0", "1"].xy.cost)
print(fgwp["0", "1"].xy.tag)

In [ ]:
# The :attr:`moscot.solvers.TaggedArray.tag` is of type :class:`moscot.solvers.Tag` and
# defines what kind of data is stored in the :class:`moscot.solvers.TaggedArray`.
# Possible tags are "cost_matrix", "kernel", and "point_cloud". Whenever the `tag` is "point_cloud",
# the backend is expted to compute the cost on the fly. Note that this often reduces
# memory complexity from quadratic to linear and hence is advisable.
# :attr:`moscot.solvers.TaggedArray.cost` should then specify which cost to compute from the
# point clouds.
#
# If the :class:`moscot.solvers.TaggedArray` corresponds to a linear term,
# :attr:`moscot.solvers.TaggedArray.data_src` and :attr:`moscot.solvers.TaggedArray.data_tgt`
# contain the point clouds of the source and the target distribution, respectively.

print(type(fgwp["0", "1"].xy.data_src))
print(type(fgwp["0", "1"].xy.data_tgt))

In [ ]:
# If the :class:`moscot.solvers.TaggedArray` corresponds to a quadratic term, the cost
# will be computed pairwise between points of the same distribution. Hence,
# :attr:`moscot.solvers.TaggedArray.data_tgt` will be `None`.

print(type(fgwp["0", "1"].x.data_src))
print(type(fgwp["0", "1"].x.data_tgt))

Whenever the [tag]{.title-ref} is \"cost\_matrix\", the backend expects
an instantiated cost matrix. There are two different cases to
distinguish. First, the user might directly want to pass custom cost
matrices, see for example
`sphx_glr_auto_examples_problems_ex_passing_custom_cost_matrices.py`{.interpreted-text
role="ref"}. In this case,
`moscot.solvers.TaggedArray.cost`{.interpreted-text role="attr"} must be
set to [custom]{.title-ref}. When setting custom cost matrices, e.g. via
`moscot.problems.base.OTProblem.set_xy`{.interpreted-text role="meth"},
the `moscot.solvers.TaggedArray`{.interpreted-text role="class"} will
change its `moscot.solvers.TaggedArray.tag`{.interpreted-text
role="attr"}. Before setting the custom cost matrix we still have
\"point\_cloud\" as a tag and
`moscot.solvers.TaggedArray.data_tgt`{.interpreted-text role="attr"} is
not [None]{.title-ref}, as it contains the point cloud of the target
distribution.


In [ ]:
print(fgwp["0", "1"].xy.tag)
print(type(fgwp["0", "1"].xy.data_tgt))

We now construct a (random) custom cost matrix for the linear term.


In [ ]:
obs_names_0 = fgwp["0", "1"].adata_src.obs_names
obs_names_1 = fgwp["0", "1"].adata_tgt.obs_names
cost_linear_01 = np.abs(rng.normal(size=(len(obs_names_0), len(obs_names_1))))
cm_linear = pd.DataFrame(data=cost_linear_01, index=obs_names_0, columns=obs_names_1)

fgwp["0", "1"].set_xy(cm_linear, tag="cost_matrix")

print(fgwp["0", "1"].xy.tag)
print(type(fgwp["0", "1"].xy.data_tgt))

If the cost matrix is to be computed via a class in
`moscot.costs`{.interpreted-text role="mod"}, the
`moscot.solvers.TaggedArray.cost`{.interpreted-text role="attr"} must be
set to the `str`{.interpreted-text role="obj"}, see for example
`sphx_glr_auto_examples_problems_ex_use_leaf_distance.py`{.interpreted-text
role="ref"}.
